In [147]:
import  pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [148]:
df = pd.read_csv('click_action_path_raw.csv')

In [35]:
df.head()

,active.user_id,log.bury_id,click.description,bury_time,rnk
0,71325658,click_01_17_002,D4,2021/6/1 00:19,1
1,71325658,click_08_09_001,A30,2021/6/1 00:20,2
2,71325658,click_08_07_015,A16,2021/6/1 00:21,3
3,71325658,click_01_18_001,D6,2021/6/2 09:39,4
4,71325658,click_08_09_001,A30,2021/6/2 09:39,5


In [129]:
df.pivot(index='active.user_id',columns='rnk',values='click.description').head(10)

rnk,1,2,3,4,5,6,7,8,9,10,...,220,221,222,223,224,225,226,227,228,229
active.user_id,,,,,,,,,,,,,,,,,,,,,
71325658,D4,A30,A16,D6,A30,A2,A30,A30,A2,A2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325671,D4,A30,A30,A37,B10,F1,D6,B10,A32,A30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325754,A57,E1,B10,A32,A30,A7,A31,A30,A7,A21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325794,D4,B10,b15,A32,A30,E1,A32,B10,b14,B13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325820,D4,B10,A32,A30,A30,A49,A31,A30,A30,A31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325839,D4,B10,B10,E1,E1,B10,E1,A32,E1,E1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71326239,D4,c5,F1,B10,A32,F1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71326439,D4,A42,F1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71326780,D4,A58,A59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
df1 = df.pivot(index='active.user_id',columns='rnk',values='click.description')
df1.describe()

rnk,1,2,3,4,5,6,7,8,9,10,...,220,221,222,223,224,225,226,227,228,229
count,11334,11229,11104,10925,10751,10550,10286,10019,9719,9365,...,1,1,1,1,1,1,1,1,1,1
unique,66,76,83,84,88,90,84,89,89,84,...,1,1,1,1,1,1,1,1,1,1
top,D4,A30,A30,A32,A32,A32,A30,A30,A32,A30,...,A1,A9,A16,A1,A7,A2,A2,A2,A2,A1
freq,6326,2863,1384,1290,1324,1216,1136,1078,988,958,...,1,1,1,1,1,1,1,1,1,1


In [131]:
df1[df1.index==71325658]

rnk,1,2,3,4,5,6,7,8,9,10,...,220,221,222,223,224,225,226,227,228,229
active.user_id,,,,,,,,,,,,,,,,,,,,,
71325658,D4,A30,A16,D6,A30,A2,A30,A30,A2,A2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
series = df1[df1.columns[0:]].apply(
    lambda x: ''.join(x.dropna().astype(str)),
    axis=1
)

In [134]:
series.head()

active.user_id
71325658    D4A30A16D6A30A2A30A30A2A2A58A30A2A2A9A9A9A30A2...
71325671    D4A30A30A37B10F1D6B10A32A30A31A30A16A9A9A9A9A9...
71325754         A57E1B10A32A30A7A31A30A7A21A7A49A50A30A7A2D6
71325794    D4B10b15A32A30E1A32B10b14B13B16b14B15D6A30A14A...
71325820                     D4B10A32A30A30A49A31A30A30A31A30
dtype: object

In [43]:
series.index

Int64Index([71325658, 71325671, 71325754, 71325794, 71325820, 71325839,
            71326239, 71326439, 71326780, 71326965,
            ...
            72106653, 72106669, 72106762, 72106793, 72106808, 72106840,
            72106902, 72106913, 72106991, 72107059],
           dtype='int64', name='active.user_id', length=11334)

## Get n-Gram

In [113]:
import re
def get_ngrams(query, n):
    tempQuery = str(query)
    a=re.findall('([A-z][0-9]?[0-9])',tempQuery)
    ngrams = []
    for i in range(0,len(a)-(n-1)):
        ngrams.append("".join(a[i:i+n]))
    return ngrams

In [167]:
gram = []
for i in series.index:
    ngrams = get_ngrams(series[i],8) ##change n-gram!!
    gram.append(ngrams)
series_3gram = pd.Series(gram,index=series.index)

In [168]:
d={}
for i in series.index:
    c = dict(Counter(series_3gram[i]))
    total = ''
    for key in c.keys():
        part = str(key)+'('+str(c[key])+')'
        total += part
    d[i]=total

In [169]:
list(d.items())[:2]

[(71325658,
  'D4A30A16D6A30A2A30A30(1)A30A16D6A30A2A30A30A2(1)A16D6A30A2A30A30A2A2(1)D6A30A2A30A30A2A2A58(1)A30A2A30A30A2A2A58A30(1)A2A30A30A2A2A58A30A2(1)A30A30A2A2A58A30A2A2(1)A30A2A2A58A30A2A2A9(1)A2A2A58A30A2A2A9A9(1)A2A58A30A2A2A9A9A9(1)A58A30A2A2A9A9A9A30(1)A30A2A2A9A9A9A30A2(1)A2A2A9A9A9A30A2A2(1)A2A9A9A9A30A2A2A1(1)A9A9A9A30A2A2A1c12(1)A9A9A30A2A2A1c12A30(1)A9A30A2A2A1c12A30A2(1)A30A2A2A1c12A30A2A9(1)A2A2A1c12A30A2A9A9(1)A2A1c12A30A2A9A9B10(1)A1c12A30A2A9A9B10b15(1)c12A30A2A9A9B10b15b14(1)A30A2A9A9B10b15b14B12(1)A2A9A9B10b15b14B12b31(1)'),
 (71325671,
  'D4A30A30A37B10F1D6B10(1)A30A30A37B10F1D6B10A32(1)A30A37B10F1D6B10A32A30(1)A37B10F1D6B10A32A30A31(1)B10F1D6B10A32A30A31A30(1)F1D6B10A32A30A31A30A16(1)D6B10A32A30A31A30A16A9(1)B10A32A30A31A30A16A9A9(1)A32A30A31A30A16A9A9A9(1)A30A31A30A16A9A9A9A9(1)A31A30A16A9A9A9A9A9(1)A30A16A9A9A9A9A9A9(1)A16A9A9A9A9A9A9A9(1)A9A9A9A9A9A9A9A9(1)A9A9A9A9A9A9A9c13(1)A9A9A9A9A9A9c13c5(1)A9A9A9A9A9c13c5C5(1)A9A9A9A9c13c5C5C5(1)A9A9A9c13c5C5C5C5(1)A9

## Remove NAs in n-grams

In [170]:
droplist=[]
for i,j in d.items():
    grams=re.findall('([A-z][0-9]?[0-9])',j)
    if len(grams)<5: ##change n-gram here!!!!!
        droplist.append(i)
len(droplist)        

1315

In [171]:
d1=d
for i in list(d1.keys()):
    if i in droplist:
        del d1[i]

In [172]:
len(d1)

10019

In [173]:
#dir(pd.DataFrame)

In [174]:
d2=pd.DataFrame(d1.items(), columns=['user_id', 'seq']).reset_index()
d2['index_1']=d2['index']+1
d3=d2[['index_1','seq']]

In [175]:
d3.to_csv("myfile_click8.txt",sep='\t', index=False, header=False)